In [2]:
import pandas as pd
import numpy as np
from statsmodels.api import OLS, add_constant

In [14]:
import gurobipy as gp
from gurobipy import GRB

class InfeasibleModel(Exception):
    pass

def optimize_monotonic_schedule(d60):
    prices = [60, 54, 48, 36]
    demand = {60: d60, 54: 1.30*d60, 48:1.79*d60, 36:2.81*d60}
    T = range(1,16)

    m = gp.Model()
    y = m.addVars(T, prices, vtype=GRB.BINARY, name="y")

    # one price/week
    for t in T:
        m.addConstr(y.sum(t,"*") == 1)
    m.addConstr(y[1,60] == 1)  # week-1 at $60
    m.addConstr(
        gp.quicksum(demand[p]*y[t,p] for t in T for p in prices)
        <= 2000
    )
    # monotonicity
    idx = {60:1, 54:2, 48:3, 36:4}
    for t in range(1,15):
        for j in (1,2,3):
            m.addConstr(
                gp.quicksum(y[t,p]   for p in prices if idx[p]<=j)
              >= gp.quicksum(y[t+1,p] for p in prices if idx[p]<=j)
            )
    # objective
    m.setObjective(
        gp.quicksum(p * demand[p] * y[t,p] for t in T for p in prices),
        GRB.MAXIMIZE
    )

    m.optimize()
    if m.status != GRB.OPTIMAL:
        # infeasible or unbounded
        raise InfeasibleModel(f"Model status {m.status}")

    # extract the 1-week assignments using .X
    schedule = {
        (t,p): 1
        for t in T for p in prices
        if y[t,p].X > 0.5
    }
    return schedule, m.ObjVal



In [15]:
#Based on that result can you create a dictionary, in which the keys are the prices and the values the number of times that price is used. It should receive as an input the function created above.
def create_price_dict(schedule):
    price_dict = {}
    for (t, p), value in schedule.items():
        if value > 0.5:  # Only consider prices that are used
            if p not in price_dict:
                price_dict[p] = 0
            price_dict[p] += 1
    return price_dict
#Example usage
schedule, _ = optimize_monotonic_schedule(100)
price_dict = create_price_dict(schedule)
print(price_dict)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.4.0 24E263)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 59 rows, 60 columns and 289 nonzeros
Model fingerprint: 0xfff5a00c
Variable types: 0 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [6e+03, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 33 rows and 34 columns
Presolve time: 0.00s
Presolved: 26 rows, 26 columns, 92 nonzeros
Variable types: 0 continuous, 26 integer (26 binary)
Found heuristic solution: objective 100824.00000
Found heuristic solution: objective 105852.00000

Root relaxation: objective 1.068465e+05, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 106846

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

def simulate_game_with_auto_demand(headless: bool = False):
    """
    1) Launch game
    2) Read Week-1 sales → d60
    3) Optimize schedule → price_dict
    4) Play weeks 2–15 following price_dict
    5) Return d60, price_dict, revenue, perfect, score
    """
    # 1) start browser
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    wait = WebDriverWait(driver, 10)
    driver.get("http://www.randhawa.us/games/retailer/nyu.html")

    # locate controls
    maintain = wait.until(EC.element_to_be_clickable((By.ID,   "maintainButton")))
    dc_10    = wait.until(EC.element_to_be_clickable((By.ID,   "tenButton")))
    dc_20    = wait.until(EC.element_to_be_clickable((By.ID,   "twentyButton")))
    dc_40    = wait.until(EC.element_to_be_clickable((By.ID,   "fortyButton")))

    # 2) read Week-1 sales
    rows = wait.until(EC.presence_of_all_elements_located((
        By.XPATH, '//table[@id="result-table"]//tr'
    )))
    first_cells = rows[-1].find_elements(By.TAG_NAME, "td")
    d60 = int(first_cells[-2].text)

    # 3) optimize & build dictionary
    schedule, _   = optimize_monotonic_schedule(d60)
    price_dict    = create_price_dict(schedule)

    # 4) play weeks 2–15 by blocks
    #  4a) first, maintain at $60 for (price_dict[60]−1) extra weeks
    for _ in range(price_dict.get(60, 0) - 1):
        maintain.click()

    #  4b) then for each discount block:
    discount_btn = {54: dc_10, 48: dc_20, 36: dc_40}
    for price in (54, 48, 36):
        weeks = price_dict.get(price, 0)
        if weeks > 0:
            discount_btn[price].click()
            for _ in range(weeks - 1):
                maintain.click()

    # 5) grab final metrics
    rev_txt     = wait.until(EC.presence_of_element_located((By.ID, "rev"))).text
    perfect_txt = wait.until(EC.presence_of_element_located((By.ID, "perfect"))).text

    revenue = int(re.sub(r"[^\d]", "", rev_txt))
    perfect = int(re.sub(r"[^\d]", "", perfect_txt))
    score   = 1 - revenue / perfect

    driver.quit()

    return {
        "d60":        d60,
        "price_dict": price_dict,
        "revenue":    revenue,
        "perfect":    perfect,
        "score":      score
    }


# --- Example usage ---
if __name__ == "__main__":
    result = simulate_game_with_auto_demand(headless=True)
    print("Week-1 demand (d60):", result["d60"])
    print("Weeks per price:",    result["price_dict"])
    print(f"Final score: {result['score']:.4f}")
    print(f"Revenue:    {result['revenue']} / Perfect: {result['perfect']}")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.4.0 24E263)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 59 rows, 60 columns and 289 nonzeros
Model fingerprint: 0x9b793324
Variable types: 0 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e+03, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 62962.560000
Presolve removed 15 rows and 16 columns
Presolve time: 0.00s
Presolved: 44 rows, 44 columns, 184 nonzeros
Variable types: 0 continuous, 44 integer (44 binary)
Found heuristic solution: objective 65761.440000

Root relaxation: objective 8.927944e+04, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 89279

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import re
import statsmodels.stats.api as sms

def batch_simulate_optimal(n_runs=20, headless=False):
    """
    Runs the retailer game n_runs times.  On each run:
      1) Read week-1 sales → d60
      2) Call optimize_monotonic_schedule(d60)
         - on InfeasibleModel: skip this run
      3) Build price_dict via create_price_dict
      4) Play weeks 2–15 according to price_dict
      5) Scrape final revenue & perfect → score
      6) Restart and repeat

    Returns a dict containing per-run data plus summary stats.
    """
    # 1) Selenium setup
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    wait = WebDriverWait(driver, 10)
    driver.get("http://www.randhawa.us/games/retailer/nyu.html")

    # 2) Locate buttons once
    maintain = wait.until(EC.element_to_be_clickable((By.ID,   "maintainButton")))
    dc_10    = wait.until(EC.element_to_be_clickable((By.ID,   "tenButton")))
    dc_20    = wait.until(EC.element_to_be_clickable((By.ID,   "twentyButton")))
    dc_40    = wait.until(EC.element_to_be_clickable((By.ID,   "fortyButton")))
    restart  = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))

    # 3) Prepare collectors
    demands, schedules, revenues, perfects, scores = [], [], [], [], []
    skipped = 0

    for _ in range(n_runs):
        # 4) Read Week-1 sales → d60
        rows = wait.until(EC.presence_of_all_elements_located((
            By.XPATH, '//table[@id="result-table"]//tr'
        )))
        d60 = int(rows[-1].find_elements(By.TAG_NAME, "td")[-2].text)

        # 5) Optimize; skip if infeasible
        try:
            schedule, _ = optimize_monotonic_schedule(d60)
        except InfeasibleModel:
            skipped += 1
            restart.click()
            continue

        # 6) Build price_dict and record demand
        price_dict = create_price_dict(schedule)
        demands.append(d60)
        schedules.append(price_dict)

        # 7) Play weeks 2–15:
        #    a) Maintain at $60 for the remaining weeks in its block
        for __ in range(price_dict.get(60, 0) - 1):
            maintain.click()

        #    b) Then for each discount block, in descending price order
        for price, btn in [(54, dc_10), (48, dc_20), (36, dc_40)]:
            weeks = price_dict.get(price, 0)
            if weeks > 0:
                btn.click()
                for __ in range(weeks - 1):
                    maintain.click()

        # 8) Scrape final metrics
        rev_txt  = wait.until(EC.presence_of_element_located((By.ID, "rev"))).text
        perf_txt = wait.until(EC.presence_of_element_located((By.ID, "perfect"))).text
        revenue  = int(re.sub(r"[^\d]", "", rev_txt))
        perfect  = int(re.sub(r"[^\d]", "", perf_txt))
        score    = 1 - revenue / perfect

        revenues.append(revenue)
        perfects.append(perfect)
        scores.append(score)

        # 9) Restart for next iteration
        restart.click()

    driver.quit()

    # 10) Compute summary stats (only on successful runs)
    mean_score = np.mean(scores) if scores else None
    std_score  = np.std(scores, ddof=1) if len(scores) > 1 else None
    if len(scores) > 1:
        ci_low, ci_high = sms.DescrStatsW(scores).tconfint_mean()
    else:
        ci_low = ci_high = None

    return {
        "n_requested":  n_runs,
        "n_skipped":    skipped,
        "demands":      demands,
        "schedules":    schedules,
        "revenues":     revenues,
        "perfects":     perfects,
        "scores":       scores,
        "mean_score":   mean_score,
        "std_score":    std_score,
        "95%_CI_score": (ci_low, ci_high),
    }


In [21]:
if __name__ == "__main__":
    summary = batch_simulate_optimal(n_runs=100, headless=True)
    print(f"Requested={summary['n_requested']}, Skipped={summary['n_skipped']}")
    print(f"Mean score:   {summary['mean_score']:.4f}")
    print(f"95% CI score: ({summary['95%_CI_score'][0]:.4f}, {summary['95%_CI_score'][1]:.4f})")
    print("Sample schedule from run 1:", summary["schedules"][0])


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.4.0 24E263)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 59 rows, 60 columns and 289 nonzeros
Model fingerprint: 0x6e19ce7c
Variable types: 0 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e+03, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 67560.120000
Presolve removed 14 rows and 14 columns
Presolve time: 0.00s
Presolved: 45 rows, 46 columns, 193 nonzeros
Variable types: 0 continuous, 46 integer (46 binary)
Found heuristic solution: objective 70317.840000

Root relaxation: objective 8.840697e+04, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 88406